In [0]:
%run "/Workspace/Users/damirgafic@gmail.com/citibike_project/citibike_project2/dev/files/src/citibike/citibike_utils"

In [0]:
%run "../../../src/utils/datetime_utils"

In [0]:

from pyspark.sql.functions import create_map, lit

In [0]:
dbutils.widgets.text(
    "pipeline_id",
    "",
    "Pipeline ID"
)
dbutils.widgets.text(
    "run_id",
    "",
    "Run ID"
)
dbutils.widgets.text(
    "task_id",
    "",
    "Task ID"
)
dbutils.widgets.text(
    "processed_timestamp",
    "",
    "Processed Timestamp"
)
dbutils.widgets.text(
    "catalog",
    "",
    "Catalog"
)

pipeline_id = dbutils.widgets.get("pipeline_id")
run_id = dbutils.widgets.get("run_id")
task_id = dbutils.widgets.get("task_id")
processed_timestamp = dbutils.widgets.get("processed_timestamp")
catalog = dbutils.widgets.get("catalog")

In [0]:
df = spark.read.table(f"{catalog}.01_bronze.jc_citibike")

In [0]:
df = get_trip_duration_mins(spark, df, "started_at", "ended_at", "trip_duration_mins")

In [0]:
df = timestamp_to_date_col(spark, df, "started_at", "trip_start_date")

In [0]:

df = df.withColumn("metadata", 
              create_map(
                  lit("pipeline_id"), lit(pipeline_id),
                  lit("run_id"), lit(run_id),
                  lit("task_id"), lit(task_id),
                  lit("processed_timestamp"), lit(processed_timestamp)
                  ))

In [0]:
df = df.select(
    "ride_id",
    "trip_start_date",
    "started_at",
    "ended_at",
    "start_station_name",
    "end_station_name",
    "trip_duration_mins",
    "metadata"
    )



In [0]:
df.write.\
    mode("overwrite").\
    option("overwriteSchema", "true").\
    saveAsTable(f"citibike_dev.02_silver.jc_citibike")